In [ ]:
%pip install -q -U transformers peft accelerate optimum
%pip install bitsandbytes
#!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/
%pip install nltk
%git clone https://github.com/PanQiWei/AutoGPTQ
%cd AutoGPTQ
%pip install .
%cd ..
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig

# Caricamento del modello base

base_model_id = "TheBloke/Mistral-7B-v0.1-GPTQ"
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)
model = AutoModelForCausalLM.from_pretrained(base_model_id,quantization_config=quantization_config_loading, device_map="auto", trust_remote_code=True)
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

# Prompt e frase di input
question = "An extension of our method to space-time interest point detection for action classification is also presented"
eval_prompt = f"""### Question: Identify and extract without rephrasing all entities that belongs to the categories Method, Metric, Material, Task, OtherScientificTerm, Generic from the following sentence: "{question}"
Write the answer in this format: Metric: entity, entity; Method: entity, entity; Material: entity, entity; Task: entity, entity; OtherScientificTerm: entity, entity; Generic: entity, entity;
These are the annotation guideline to extract entities:
Task are Applications, problems to solve, systems to construct;
Method are Methods , models, systems to use, or tools, components of a system;
Evaluation Metric are Metrics, measures, or entities that can express quality of a system/method;
Material are Data, datasets, resources, Corpus, Knowledge base;
OtherScientificTerms are Phrases that are a scientific terms but do not fall into any of the above classes;
Generic are General terms or pronouns that may refer to a entity but are not themselves informative.
### Answer:"""


# Caricamento del modello fine-tuned
from peft import PeftModel

ft_model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/Tesi/models/mistral-ner-finetune-def/checkpoint-200")


model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
  result = eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens= 1020,do_sample = False)[0], skip_special_tokens=True)
  print(result)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPTQConfig

# Caricamento del modello base
base_model_id = "TheBloke/Mistral-7B-v0.1-GPTQ"
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True)
model = AutoModelForCausalLM.from_pretrained(base_model_id,quantization_config=quantization_config_loading, device_map="auto", trust_remote_code=True)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

# Caricamento del modello fine-tuned
from peft import PeftModel

ft_model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/Tesi/models/mistral-ner-finetune-new-prompt/checkpoint-348")
ft_model.eval()


# Path del file di output
path_result = "/content/drive/MyDrive/Tesi/result/result_mistral_new_prompt.json"
path_dataset = "/content/drive/MyDrive/Tesi/dataset/scierc_test_inc.json"

import json

# Per ogni riga del dataset si effettua la predizione e si scrive il risultato nel file di output
with open(path_dataset) as f:
    for line in f:
        data = json.loads(line)
        question = data["input"]
        eval_prompt = f"""### Question: Identify and extract without rephrasing all entities that belongs to the categories Method, Metric, Material, Task, OtherScientificTerm, Generic from the following sentence: "{question}"
Write the answer in this format: Metric: entity, entity; Method: entity, entity; Material: entity, entity; Task: entity, entity; OtherScientificTerm: entity, entity; Generic: entity, entity;
These are the annotation guideline to extract entities:
Task are Applications, problems to solve, systems to construct;
Method are Methods , models, systems to use, or tools, components of a system;
Evaluation Metric are Metrics, measures, or entities that can express quality of a system/method;
Material are Data, datasets, resources, Corpus, Knowledge base;
OtherScientificTerms are Phrases that are a scientific terms but do not fall into any of the above classes;
Generic are General terms or pronouns that may refer to a entity but are not themselves informative.
### Answer:"""
        model_input = eval_tokenizer(eval_prompt, return_tensors="pt").input_ids.cuda()
        result = eval_tokenizer.decode(ft_model.generate(inputs = model_input, max_new_tokens= 200,do_sample = False)[0], skip_special_tokens=True)
        with open(path_result, 'a') as outfile:
          json_object = {"prediction": result}
          json.dump(json_object, outfile)
          outfile.write('\n')
